#### Imports

In [5]:
from utils.game import *
from utils.agent import *
from utils.world import *
from utils.viz import *
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json


def read_ready_map_from_json(path):
    with open(path, 'r') as f:
        map_dict = json.load(f)
    
    map_graph = np.array(map_dict["map_graph"])
    layout = map_dict["layout"]
    countries = map_dict["countries"]
    nb_territories = len(countries)

    colors = map_dict["colors"]
    nb_players = map_dict["nb_players"]
    labels = map_dict["labels"]
    presence_map_start = np.array(map_dict["presence_map_start"])

    # Change countries  and layout keys to integers
    countries = {int(k): v for k, v in countries.items()}
    layout = {int(k): v for k, v in layout.items()}

    return map_graph, layout, countries, nb_territories, colors, nb_players, labels, presence_map_start


def read_empty_map_from_json(path):
    with open(path, 'r') as f:
        map_dict = json.load(f)
    
    map_graph = np.array(map_dict["map_graph"])
    layout = map_dict["layout"]
    countries = map_dict["countries"]
    nb_territories = len(countries)

    # Change countries  and layout keys to integers
    countries = {int(k): v for k, v in countries.items()}
    layout = {int(k): v for k, v in layout.items()}

    return map_graph, layout, countries, nb_territories


#### Run

In [8]:
# Read map from json
maps = ["./maps/map_5nodes_3p_Continents.json", "./maps/map_11nodes_3p_Europe1914.json"]  
map_ = maps[0]
map_graph, layout, countries, nb_territories, colors, nb_player, labels, presence_map_start = read_ready_map_from_json(map_)


Run the game as if the NaiveAgent was playing

In [6]:
naive_agent = NaiveAgent(nb_territories=nb_territories)

# Start
game = Game(map_graph=map_graph, presence_map=presence_map_start, nb_players=nb_player)
print("Presence map:\n", game.world.presence_map)
print("Player 0 territories: ", game.world.get_territories(0))

player_presence_map = game.world.get_player_presence_map(p=0)
print("Player 0 presence map: ", player_presence_map)

# Reinforcements
to_deploy = game.world.get_reinforcements(p=0)
print("Reinforcements for player 0: ", to_deploy)

# Deployment
t = naive_agent.choose_deploy(to_deploy, player_presence_map)
print("Chosen territory: ", t)
game.world.deploy(p=0, t=t, n=to_deploy)
print("Presence map after deployment:\n", game.world.presence_map)

# Attack
player_presence_map = game.world.get_player_presence_map(p=0)
attacks = game.world.get_available_targets(p=0)
print("Possible attacks: ", attacks)
if len(attacks) > 0:
    chosen_attack = naive_agent.choose_attack(attacks, player_presence_map)
    player_attacked = game.world.get_owner(chosen_attack[1])
    print("Chosen attack: ", chosen_attack, " ; player attacked: ", player_attacked)
    game.resolve_battle(player1=0, player2=player_attacked,
                        t_orig=chosen_attack[0], t_dest=chosen_attack[1])
    print("Presence map after attack:\n", game.world.presence_map)

# Fortification
player_presence_map = game.world.get_player_presence_map(p=0)
available_fortifications = game.world.get_available_fortifications(p=0)
print("Available fortifications: ", available_fortifications)
if len(available_fortifications) > 0:
    chosen_fortification = naive_agent.choose_fortify(available_fortifications, player_presence_map)
    print("Chosen fortification: ", chosen_fortification)
    game.world.fortify(p=0, t_orig=0, t_dest=1)
    print("Presence map after fortification:\n", game.world.presence_map)


Presence map:
 [[2 2 0 2 0 0 0 0 1 0 0]
 [0 0 2 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 2 0 2 0 2 2]]
Player 0 territories:  [0, 1, 3, 8]
Player 0 presence map:  [ 2  2 -2  2 -2 -2 -2 -2  1 -2 -2]
Reinforcements for player 0:  12
Chosen territory:  0
Presence map after deployment:
 [[14  2  0  2  0  0  0  0  1  0  0]
 [ 0  0  2  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  2  0  2  2]]
Possible attacks:  [(1, 2), (3, 4), (1, 5), (0, 9), (3, 10), (1, 7), (3, 9), (3, 6), (3, 2)]
Chosen attack:  (1, 2)  ; player attacked:  1
Presence map after attack:
 [[14  1  1  2  0  0  0  0  1  0  0]
 [ 0  0  0  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  2  0  2  2]]
Available fortifications:  [(0, 1), (0, 8), (3, 2)]
Chosen fortification:  (0, 1)
Presence map after fortification:
 [[ 1 14  1  2  0  0  0  0  1  0  0]
 [ 0  0  0  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  2  0  2  2]]


Test to run some turns with naive agents and save the maps

In [7]:
# Start
game = Game(map_graph=map_graph, presence_map=presence_map_start, nb_players=nb_player,
            countries=countries, colors=colors, layout=layout)
game.agents = [RandomAgent(nb_territories=nb_territories) for _ in range(nb_player)]

# Play some turns
game.run_and_save()

# Generate a gif and save it
# generate_gif("./images/", "./gifs/", fps=0.5)

c:\Users\chiro\Documents\Projets\Risk_RL\utils\viz.py:65: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 6))
c:\Users\chiro\Documents\Projets\Risk_RL\utils\viz.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

Try reinforce

In [8]:
game = Game(map_graph=map_graph, presence_map=presence_map_start, nb_players=nb_player,
            countries=countries, colors=colors, layout=layout)
game.agents = [PolicyGradientAgent(nb_territories=nb_territories),
               RandomAgent(nb_territories=nb_territories),
               RandomAgent(nb_territories=nb_territories)]

test_agent = PolicyGradientAgent(nb_territories=nb_territories)

print("Presence map:\n", game.world.presence_map)
print("Player 0 territories: ", game.world.get_territories(0))

player_presence_map = torch.tensor(game.world.get_player_presence_map(p=0)).float()
print("Player 0 presence map: ", player_presence_map)

# Reinforcements
to_deploy = torch.tensor([game.world.get_reinforcements(p=0)])
print("Reinforcements for player 0: ", to_deploy)

# Deployment
t, prob = test_agent.choose_deploy_prob(to_deploy, player_presence_map)
print("Chosen territory: ", t, " ; prob: ", prob)
game.world.deploy(p=0, t=t, n=to_deploy)
print("Presence map after deployment:\n", game.world.presence_map)

# Attack
player_presence_map = torch.tensor(game.world.get_player_presence_map(p=0)).float()
attacks = game.world.get_available_targets(p=0)
print("Possible attacks: ", attacks)
if len(attacks) > 0:
    t_orig, t_dest, prob = test_agent.choose_attack_prob(attacks, player_presence_map)
    player_attacked = game.world.get_owner(t_dest)
    print("Chosen attack: ", t_orig, t_dest, " ; player attacked: ", player_attacked, " ; prob: ", prob)
    game.resolve_battle(player1=0, player2=player_attacked,
                        t_orig=t_orig, t_dest=t_dest)
    print("Presence map after attack:\n", game.world.presence_map)

# Fortification
player_presence_map = torch.tensor(game.world.get_player_presence_map(p=0)).float()
available_fortifications = game.world.get_available_fortifications(p=0)
print("Available fortifications: ", available_fortifications)
if len(available_fortifications) > 0:
    t_orig, t_dest, prob = test_agent.choose_fortify_prob(available_fortifications, player_presence_map)
    print("Chosen fortification: ", t_orig, t_dest, " ; prob: ", prob)
    game.world.fortify(p=0, t_orig=t_orig, t_dest=t_dest)
    print("Presence map after fortification:\n", game.world.presence_map)


Presence map:
 [[2 2 0 2 0 0 0 0 1 0 0]
 [0 0 2 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 2 0 2 0 2 2]]
Player 0 territories:  [0, 1, 3, 8]
Player 0 presence map:  tensor([ 2.,  2., -2.,  2., -2., -2., -2., -2.,  1., -2., -2.])
Reinforcements for player 0:  tensor([12])
Chosen territory:  0  ; prob:  tensor([-1.2361], grad_fn=<SqueezeBackward1>)
Presence map after deployment:
 [[14  2  0  2  0  0  0  0  1  0  0]
 [ 0  0  2  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  2  0  2  2]]
Possible attacks:  [(1, 2), (3, 4), (1, 5), (0, 9), (3, 10), (1, 7), (3, 9), (3, 6), (3, 2)]
Chosen attack:  3 6  ; player attacked:  1  ; prob:  tensor([-2.2141], grad_fn=<SqueezeBackward1>)
Presence map after attack:
 [[14  2  0  1  0  0  0  0  1  0  0]
 [ 0  0  2  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  2  0  2  2]]
Available fortifications:  [(0, 1), (0, 8), (1, 0), (1, 8)]
Chosen fortification:  0 1  ; prob:  tensor([-1.4153], grad_fn=<SqueezeBackward1>)
Presence map after fortification:
 [[ 1 15  0  1  0 

In [9]:
game = Game(map_graph=map_graph, presence_map=presence_map_start, nb_players=nb_player,
            countries=countries, colors=colors, layout=layout)
game.agents = [PolicyGradientAgent(nb_territories=nb_territories),
               RandomAgent(nb_territories=nb_territories),
               RandomAgent(nb_territories=nb_territories)]

print("Presence map:\n", game.world.presence_map)
for j in range(20):
    for i in range(3):
        print("Player ", i, " territories: ", len(game.world.get_territories(i)))
        if len(game.world.get_territories(i)) > 0:
            if i == 0:
                player_presence_map = torch.tensor(game.world.get_player_presence_map(p=i)).float()
                print("Player 0 presence map: ", player_presence_map)

                # Reinforcements
                to_deploy = torch.tensor([game.world.get_reinforcements(p=i)])
                print("Reinforcements for player 0: ", to_deploy)

                # Deployment
                t, prob = game.agents[i].choose_deploy_prob(to_deploy, player_presence_map)
                print("Chosen territory: ", t, " ; prob: ", prob)
                game.world.deploy(p=i, t=t, n=to_deploy)
                print("Presence map after deployment:\n", game.world.presence_map)

                # Attack
                player_presence_map = torch.tensor(game.world.get_player_presence_map(p=i)).float()
                attacks = game.world.get_available_targets(p=0)
                print("Possible attacks: ", attacks)
                if len(attacks) > 0:
                    t_orig, t_dest, prob = game.agents[i].choose_attack_prob(attacks, player_presence_map)
                    player_attacked = game.world.get_owner(t_dest)
                    print("Chosen attack: ", t_orig, t_dest, " ; player attacked: ", player_attacked, " ; prob: ", prob)
                    game.resolve_battle(player1=i, player2=player_attacked,
                                        t_orig=t_orig, t_dest=t_dest)
                    print("Presence map after attack:\n", game.world.presence_map)

                # Fortification
                player_presence_map = torch.tensor(game.world.get_player_presence_map(p=0)).float()
                available_fortifications = game.world.get_available_fortifications(p=0)
                print("Available fortifications: ", available_fortifications)
                if len(available_fortifications) > 0:
                    t_orig, t_dest, prob = game.agents[i].choose_fortify_prob(available_fortifications, player_presence_map)
                    print("Chosen fortification: ", t_orig, t_dest, " ; prob: ", prob)
                    game.world.fortify(p=i, t_orig=t_orig, t_dest=t_dest)
                    print("Presence map after fortification:\n", game.world.presence_map)
            
            else:
                print("Player ", i)        
                print(f"Player {i} territories: ", game.world.get_territories(i))

                player_presence_map = torch.tensor(game.world.get_player_presence_map(p=i)).float()
                print(f"Player {i} presence map: ", player_presence_map)

                # Reinforcements
                to_deploy = torch.tensor([game.world.get_reinforcements(p=i)])
                print(f"Reinforcements for player {i}: ", to_deploy)

                # Deployment
                t = game.agents[i].choose_deploy(to_deploy, player_presence_map)
                print("Chosen territory: ", t)
                game.world.deploy(p=i, t=t, n=to_deploy)
                print("Presence map after deployment:\n", game.world.presence_map)

                # Attack
                player_presence_map = torch.tensor(game.world.get_player_presence_map(p=i)).float()
                attacks = game.world.get_available_targets(p=i)
                print("Possible attacks: ", attacks)
                if len(attacks) > 0:
                    t_orig, t_dest = test_agent.choose_attack(attacks, player_presence_map)
                    player_attacked = game.world.get_owner(t_dest)
                    print("Chosen attack: ", t_orig, t_dest, " ; player attacked: ", player_attacked)
                    game.resolve_battle(player1=i, player2=player_attacked,
                                        t_orig=t_orig, t_dest=t_dest)
                    print("Presence map after attack:\n", game.world.presence_map)

                # Fortification
                player_presence_map = torch.tensor(game.world.get_player_presence_map(p=i)).float()
                available_fortifications = game.world.get_available_fortifications(p=i)
                print("Available fortifications: ", available_fortifications)
                if len(available_fortifications) > 0:
                    t_orig, t_dest = test_agent.choose_fortify(available_fortifications, player_presence_map)
                    print("Chosen fortification: ", t_orig, t_dest)
                    game.world.fortify(p=i, t_orig=t_orig, t_dest=t_dest)
                    print("Presence map after fortification:\n", game.world.presence_map)

        else:
            print("Player ", i, " has lost the game")

Presence map:
 [[2 2 0 2 0 0 0 0 1 0 0]
 [0 0 2 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 2 0 2 0 2 2]]
Player  0  territories:  4
Player 0 presence map:  tensor([ 2.,  2., -2.,  2., -2., -2., -2., -2.,  1., -2., -2.])
Reinforcements for player 0:  tensor([12])
Chosen territory:  3  ; prob:  tensor([-1.1725], grad_fn=<SqueezeBackward1>)
Presence map after deployment:
 [[ 2  2  0 14  0  0  0  0  1  0  0]
 [ 0  0  2  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  2  0  2  2]]
Possible attacks:  [(1, 2), (3, 4), (1, 5), (0, 9), (3, 10), (1, 7), (3, 9), (3, 6), (3, 2)]
Chosen attack:  1 7  ; player attacked:  2  ; prob:  tensor([-2.2040], grad_fn=<SqueezeBackward1>)
Presence map after attack:
 [[ 2  1  0 14  0  0  0  1  1  0  0]
 [ 0  0  2  0  2  0  2  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  2  2]]
Available fortifications:  [(0, 1), (0, 8)]
Chosen fortification:  0 1  ; prob:  tensor([-0.7032], grad_fn=<SqueezeBackward1>)
Presence map after fortification:
 [[ 1  2  0 14  0  0  0  1  1  0  0]
 [ 0  